In [1]:
%run base.ipynb

In [2]:
Jq_ = Kinematics.Jq
J_ = Kinematics.J
J_inv = Kinematics.J_INV

In [3]:
xd = J_@x_nb
ode_xdd = uv_body.body_forward_dynamics()

rhs = vertcat(xd, ode_xdd) #the complete ODE vector
f_rhs = Function('Odefunc', [m, W, B, r_g, r_b, I_o,
                           decoupled_added_m, coupled_added_m,
                           linear_dc, quadratic_dc,
                           x_nb, v_c, eul, tau_b], [rhs])

## RETURNS vertcat(p_n ,uq , x_nb)
ode = f_rhs(ps.m, ps.W, ps.B, ps.rg, ps.rb, ps.Io, ps.added_m,
                       ps.coupl_added_m, ps.linear_dc, ps.quadratic_dc, x_nb, v_c, eul,  tau_b)

dt = SX.sym("dt")
xS0 = vertcat(n, dn)
ode_p = vertcat(dt, v_c, x_nb)

# # integrator to discretize the system
sys = {}
sys['x'] = xS0
sys['u'] = tau_b
sys['p'] = ode_p
sys['ode'] = ode*dt # Time scaling

intg = integrator('intg', 'rk', sys, 0, 1 , {'simplify':True, 'number_of_finite_elements':50})

res = intg(x0=xS0,u=tau_b, p=ode_p) #evaluate with symbols
x_next = res['xf']

x_next[6:] = J_@x_next[6:]

<h1>Generating discretized state space Linear Model </h1>

In [4]:
states = vertcat(xS0[:6], dn)

A = jacobian(x_next, states)
print(A.size())

B__ = jacobian(x_next, tau_b)
print(B__.size())

(12, 12)
(12, 6)


In [6]:
A

SX(sparse: 12-by-12, 48 nnz
 @1=1,
 @2=0.00333333,
 @3=cos(psi),
 @4=sin(thet),
 @5=(@3*@4),
 @6=sin(psi),
 @7=((@5*cos(phi))+(@6*sin(phi))),
 @8=sin(psi),
 @9=sin(thet),
 @10=cos(psi),
 @11=((@8*cos(phi))-(@9*(@10*sin(phi)))),
 @12=2,
 @13=0.01,
 @14=cos(thet),
 @15=(sin(phi)/@14),
 @16=cos(thet),
 @17=(cos(phi)/@16),
 @18=(((@15*q)+(@17*r))*dt),
 @19=(psi+(@13*@18)),
 @20=cos(@19),
 @21=cos(phi),
 @22=sin(phi),
 @23=(((@21*q)-(@22*r))*dt),
 @24=(thet+(@13*@23)),
 @25=sin(@24),
 @26=sin(phi),
 @27=cos(phi),
 @28=(dt*((q*@26)+(r*@27))),
 @29=(@13*@28),
 @30=cos(@24),
 @31=sin(@19),
 @32=(cos(phi)/@14),
 @33=(sin(phi)/@16),
 @34=(dt*((q*@32)-(r*@33))),
 @35=(@13*@34),
 @36=cos(@19),
 @37=sin(@24),
 @38=(@36*@37),
 @39=sin(phi),
 @40=tan(thet),
 @41=(@39*@40),
 @42=cos(phi),
 @43=tan(thet),
 @44=(@42*@43),
 @45=(((p+(@41*q))+(@44*r))*dt),
 @46=(phi+(@13*@45)),
 @47=cos(@46),
 @48=(@40*cos(phi)),
 @49=(@43*sin(phi)),
 @50=(dt*((q*@48)-(r*@49))),
 @51=(@1+(@13*@50)),
 @52=sin(@46),
 @53=si

In [7]:
B__

SX(sparse: 12-by-6, 26 nnz
 @1=(cos(psi)*cos(thet)),
 @2=0.00333333,
 @3=0.0594846,
 @4=2,
 @5=((((((((((((((((((((((((((((((((((((((((((((((((((@2*((((@3*dt)+(@4*(@3*dt)))+(@4*(@3*dt)))+(@3*dt)))+(@2*((((@3*dt)+(@4*(@3*dt)))+(@4*(@3*dt)))+(@3*dt))))+(@2*((((@3*dt)+(@4*(@3*dt)))+(@4*(@3*dt)))+(@3*dt))))+(@2*((((@3*dt)+(@4*(@3*dt)))+(@4*(@3*dt)))+(@3*dt))))+(@2*((((@3*dt)+(@4*(@3*dt)))+(@4*(@3*dt)))+(@3*dt))))+(@2*((((@3*dt)+(@4*(@3*dt)))+(@4*(@3*dt)))+(@3*dt))))+(@2*((((@3*dt)+(@4*(@3*dt)))+(@4*(@3*dt)))+(@3*dt))))+(@2*((((@3*dt)+(@4*(@3*dt)))+(@4*(@3*dt)))+(@3*dt))))+(@2*((((@3*dt)+(@4*(@3*dt)))+(@4*(@3*dt)))+(@3*dt))))+(@2*((((@3*dt)+(@4*(@3*dt)))+(@4*(@3*dt)))+(@3*dt))))+(@2*((((@3*dt)+(@4*(@3*dt)))+(@4*(@3*dt)))+(@3*dt))))+(@2*((((@3*dt)+(@4*(@3*dt)))+(@4*(@3*dt)))+(@3*dt))))+(@2*((((@3*dt)+(@4*(@3*dt)))+(@4*(@3*dt)))+(@3*dt))))+(@2*((((@3*dt)+(@4*(@3*dt)))+(@4*(@3*dt)))+(@3*dt))))+(@2*((((@3*dt)+(@4*(@3*dt)))+(@4*(@3*dt)))+(@3*dt))))+(@2*((((@3*dt)+(@4*(@3*dt)))+(@4*(@3*dt)))+(@3*

In [5]:
A_nb_func = Function('A_nb', [dt, x_nb, dn, eul, tau_b, v_c],[A])

current_vel = [0,0,0,0,0,0] # assuming no current flow

A_ = Function('A_', [dt, dn, eul, tau_b],[A_nb_func(dt, J_inv@(dn), dn, eul,tau_b, current_vel)])

A__ = A_(dt, dn, eul, tau_b)

next_state = A__@states + B__@tau_b

x_LTI = Function('LTI', [dt, n, dn, tau_b], [next_state, A__, B__])